In [ ]:
!pip install dash==1.10.0
!pip install dash dash-renderer dash-html-components dash-core-components plotly
!pip install plotly
!pip install dash-bootstrap-components

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math
import os
import base64
import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

In [2]:

###################################
# Private function
###################################


def load_data(file_name, column_name):
    data = (
        pd.read_csv(file_name,)
        .melt(
            id_vars=["Province/State", "Country/Region", "Lat", "Long"],
            var_name="date",
            value_name=column_name,
        )
        .astype({"date": "str", column_name: int}, errors="ignore")
    )
    data["Province/State"].fillna("<all>", inplace=True)
    data[column_name].fillna(0, inplace=True)
    return data

In [3]:

################################################################################
# Data processing
################################################################################

# Data and files working with
confirmed_cases = "time_series_covid19_confirmed_global.csv"
confirmed_deaths = "time_series_covid19_deaths_global.csv"
confirmed_recovered = "time_series_covid19_recovered_global.csv"

# Merge data frames into one
all_data = (
    load_data(confirmed_cases, "confirmed")
    .merge(load_data(confirmed_deaths, "deaths"))
    .merge(load_data(confirmed_recovered, "recovered"))
)

countries = all_data["Country/Region"].unique()
countries.sort()

In [4]:
options =[]
for country in list(countries):
    options.append({"label":country,"value":country})

# print(options)

all_data['date'] = pd.to_datetime(all_data['date'],errors='coerce')
# print(all_data.dtypes)

In [5]:
country1 = all_data[all_data['Country/Region']=="China"]
country1=country1.drop('Province/State', axis=1).groupby("date").sum().reset_index()
print(country1)

         date        Lat       Long  confirmed  deaths  recovered
0  2020-01-22  1083.3367  3684.4197        548      17         28
1  2020-01-23  1083.3367  3684.4197        643      18         30
2  2020-01-24  1083.3367  3684.4197        920      26         36
3  2020-01-25  1083.3367  3684.4197       1406      42         39
4  2020-01-26  1083.3367  3684.4197       2075      56         49
5  2020-01-27  1083.3367  3684.4197       2877      82         58
6  2020-01-28  1083.3367  3684.4197       5509     131        101
7  2020-01-29  1083.3367  3684.4197       6087     133        120
8  2020-01-30  1083.3367  3684.4197       8141     171        135
9  2020-01-31  1083.3367  3684.4197       9802     213        214
10 2020-02-01  1083.3367  3684.4197      11891     259        275
11 2020-02-02  1083.3367  3684.4197      16630     361        463
12 2020-02-03  1083.3367  3684.4197      19716     425        614
13 2020-02-04  1083.3367  3684.4197      23707     491        843
14 2020-02

In [6]:

app = dash.Dash()
app.layout = html.Div([
    html.Div(html.H1(children="COVID-19 Analysis")),

    
    html.Div(
    html.Label([dcc.Dropdown(id = "country-dropdown", options = options, placeholder = "Please select country",value="US")])
    ,style= {
              "color": "green",
              "text-align": "center","background-color": "darkorange",
              "display":"inline-block","width":"100%"
              }),
    
    html.Div(
        dcc.Graph(id="country_covid"
             )
        )

],style= {
                      "color": "black",
                      "text-align": "center","background-color": "yellow",
                      "border-style": "Bold","display":"inline-block","width":"100%",
                      })

@app.callback(
    dash.dependencies.Output("country_covid","figure"),
    [dash.dependencies.Input("country-dropdown","value")],
)
def update_fig(input_value):
    country_all_data = all_data[all_data['Country/Region']==input_value]
    country_all_data=country_all_data.drop('Province/State', axis=1).groupby("date").sum().reset_index()
#     data_fig_deaths=[]
    data_fig_confirmed=[]
    trace_line1 = go.Scatter(x=[date_obj.strftime('%Y-%m-%d') for date_obj in country_all_data["date"].dt.date.unique()],
                            y=list(country_all_data["deaths"]),
                            name="Death",
                            mode = 'lines',
                            showlegend=False)
    trace_line2 = go.Scatter(x=[date_obj.strftime('%Y-%m-%d') for date_obj in country_all_data["date"].dt.date.unique()],
                            y=list(country_all_data["confirmed"]),
                            name="Confirmed",
                            mode = 'lines',
                            showlegend=False)

    data_fig_confirmed.append(trace_line1)
    data_fig_confirmed.append(trace_line2)
    #data = [data_fig_confirmed,data_fig_deaths]
    layout = {"title":input_value}
    
    return {
        "data":data_fig_confirmed,
        "layout":layout
    }
    

if __name__=="__main__":
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
